In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['GOOGLE_API_KEY']=os.getenv('GOOGLE_API_KEY')

In [3]:
from google import genai

client = genai.Client(http_options= {
      'api_version': 'v1alpha'
})

In [4]:
model_name = "gemini-2.0-flash-exp"

In [5]:
import asyncio
import contextlib
import json
import wave

from IPython import display

from google import genai
from google.genai import types

In [6]:
@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

In [7]:

import logging
logger = logging.getLogger('Live')
#logger.setLevel('DEBUG')  # Switch between "INFO" and "DEBUG" to toggle debug messages.
logger.setLevel('INFO')

In [8]:
n = 0
async def run(prompt, modality="TEXT", tools=None):
  global n
  if tools is None:
    tools=[]

  config = {
          "tools": tools,
          "response_modalities": [modality]}

  async with client.aio.live.connect(model=model_name, config=config) as session:
    display.display(display.Markdown(prompt))
    display.display(display.Markdown('-------------------------------'))
    await session.send(input=prompt, end_of_turn=True)

    audio = False
    filename = f'audio_{n}.wav'
    with wave_file(filename) as wf:
      async for response in session.receive():
        logger.debug(str(response))
        if text:=response.text:
          display.display(display.Markdown(text))
          continue

        if data:=response.data:
          print('.', end='')
          wf.writeframes(data)
          audio = True
          continue

        server_content = response.server_content
        if server_content is not None:
          handle_server_content(wf, server_content)
          continue

        tool_call = response.tool_call
        if tool_call is not None:
          await handle_tool_call(session, tool_call)


  if audio:
    display.display(display.Audio(filename, autoplay=True))
    n = n+1

In [9]:
def handle_server_content(wf, server_content):
  model_turn = server_content.model_turn
  if model_turn:
    for part in model_turn.parts:
      executable_code = part.executable_code
      if executable_code is not None:
        display.display(display.Markdown('-------------------------------'))
        display.display(display.Markdown(f'``` python\n{executable_code.code}\n```'))
        display.display(display.Markdown('-------------------------------'))

      code_execution_result = part.code_execution_result
      if code_execution_result is not None:
        display.display(display.Markdown('-------------------------------'))
        display.display(display.Markdown(f'```\n{code_execution_result.output}\n```'))
        display.display(display.Markdown('-------------------------------'))

  grounding_metadata = getattr(server_content, 'grounding_metadata', None)
  if grounding_metadata is not None:
    display.display(
        display.HTML(grounding_metadata.search_entry_point.rendered_content))

  return

In [10]:
async def handle_tool_call(session, tool_call):
  for fc in tool_call.function_calls:
    tool_response = types.LiveClientToolResponse(
        function_responses=[types.FunctionResponse(
            name=fc.name,
            id=fc.id,
            response={'result':'ok'},
        )]
    )

    print('\n>>> ', tool_response)
    await session.send(input=tool_response)

In [12]:
await run(prompt="capital of paris?", tools=None, modality = "TEXT")

capital of paris?

-------------------------------

The

 capital of France is **Paris**.


In [13]:
turn_on_the_lights = {'name': 'turn_on_the_lights'}
turn_off_the_lights = {'name': 'turn_off_the_lights'}


prompt = "Turn on the lights"

tools = [
    {'function_declarations': [turn_on_the_lights, turn_off_the_lights]}
]

await run(prompt, tools=tools, modality = "TEXT")
     

Turn on the lights

-------------------------------

-------------------------------

``` python
print(default_api.turn_on_the_lights())

```

-------------------------------


>>>  function_responses=[FunctionResponse(id='function-call-13881645250001334133', name='turn_on_the_lights', response={'result': 'ok'})]


-------------------------------

```
{'result': 'ok'}

```

-------------------------------

OK

In [17]:
prompt="Can you compute the prime number between 1 to 100"

tools = [
    {'code_execution': {}}
]

await run(prompt, tools=tools, modality="TEXT")

Can you compute the prime number between 1 to 100

-------------------------------

-------------------------------

``` python
def is_prime(n):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

primes = [number for number in range(2, 101) if is_prime(number)]
print(f'{primes=}')

```

-------------------------------

...

 prime numbers between 1 and 100 are: 2, 

3, 5, 7, 11, 13,

 17, 19, 23, 29, 31, 37, 41, 43,

 47, 53, 59, 61, 67, 71, 73, 79,

 83, 89, and 97.


In [18]:
prompt="Can you use google search tell me about the largest earthquake in california the week of Dec 5 2024?"

tools = [
   {'google_search': {}}
]

await run(prompt, tools=tools, modality="TEXT")

Can you use google search tell me about the largest earthquake in california the week of Dec 5 2024?

-------------------------------

-------------------------------

``` python
print(google_search.search(queries=["largest earthquake in California week of December 5 2024", "California earthquakes December 2024"]))

```

-------------------------------

...

 occurred at the Mendocino Triple Junction, where the Pacific, North America, and Juan de Fuca/Gorda plates meet. The specific fault was along the Mendocino Fracture Zone, a transform boundary where the Juan de Fuca/Gorda Plate is moving eastward relative to the Pacific Plate.
*   

**Tsunami Warning:** A tsunami warning was issued for coastal areas of Northern California and Southern Oregon, including as far south as San Francisco. The warning was canceled about an hour later after it was determined that a destructive tsunami was not likely.
*   **Impacts:**
    *   Power outages were reported in

 Humboldt County, affecting over 10,000 customers.
    *   Some structural damage was reported, including cracked roads in Rio Dell and minor damage to roofs in Ferndale.
    *   There were no reports of injuries.
*   **Aftershocks:** There were numerous aftershocks

, with more than 30 occurring throughout the day. The largest aftershock measured 4.3 and occurred near Petrolia. There is a 34% chance of one or more aftershocks larger than magnitude 5 within the next week.

This earthquake was one of the largest to impact the North

 Coast since the 1992 M7.2 Cape Mendocino earthquake.
